In [ ]:
from __future__ import print_function, division,generators
import numpy as np
import numpy.ma as ma
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns
import os
import glob
from scipy.stats import norm as scipy_stats_norm
%matplotlib inline

###1. Read the station data downloaded from GHCN archive###
Data obtained from http://www.ncdc.noaa.gov/cdo-web/

In [ ]:
def get_date(date_number):
    """
    Turn the int64 value from the DATE of GHCN into a pd.datetime
    """
    dstring = str(date_number)
    return pd.datetime(int(dstring[0:4]),int(dstring[4:6]),int(dstring[6:8]))

def get_df(fnm, var, no_missing = True):
    """
    Create a dataframe for a single station, with a time index, for a single
    variable of data given as a key word (e.g. PRECIP, TMAX, TMIN).
    Requires file path and name (fnm).
    no_missing is a Bool that optionally masks out values < -99 from the df.
    """
    df = pd.read_csv(fnm)
    dt_indx = [get_date(date) for date in df.DATE]
    data_vals = df[var].values
    if var is 'PRCP':
        data_vals = data_vals / 10.  # This is to convert precip data to mm
    if no_missing:
        tmp_df = pd.DataFrame(data=data_vals,
                              index=dt_indx,columns=[df.STATION[0][6:]])
        mask = tmp_df > -99.  # A catchall value for missing data in GHCN
        return tmp_df[mask]
    else:
        return pd.DataFrame(data=data_vals,
                            index=dt_indx,columns=[df.STATION[0][6:]])

def get_combined_df(fpth, var):
    """
    From a given file path, and variable, extract data from all .csv files, and
    place in a single dataframe object.
    """
    flist = glob.glob(fpth)
    df_dic = {}
    for f in flist:
        df_dic[f[5:]] = get_df(fnm = f, var = var, no_missing=True)
    return pd.concat([df_dic[key] for key in df_dic.keys()],axis=1)

Call the Get_combined() function to create dataframes out of all data in a folder.

In [125]:
%%time
df_tmax = get_combined_df(fpth="Data/*.csv",var="TMAX")
df_tmin = get_combined_df(fpth="Data/*.csv",var="TMIN")
df_prcp = get_combined_df(fpth="Data/*.csv",var="PRCP")

CPU times: user 2.63 s, sys: 119 ms, total: 2.75 s
Wall time: 2.72 s


In [124]:
for station in df_prcp:
    print(station, np.max(df_prcp[station]))

KE000063619 167.6
ET000063403 169.9
TZ000063832 161.8
SU000062941 138.0
ET000063471 152.1
Accumulated 52.55
Acc_SEM 50.55
Acc_anomaly 47.170625


Plot time series of precipitation for all stations, and also accumulate the data and plot the average rainfall.

In [ ]:
# Example of masking and accessing data from stations...
#station = df_prcp.keys()[1]
#plt.plot(df_prcp[station].index,df_prcp[station],'.',alpha=0.5)
#plt.title("Station {0:s}".format(station))

In [ ]:
#df_prcp.KE000063740[df_prcp.KE000063740 > 100]

###2. Time series plots###

Daily Mean and SEM values: Mean uncertainty is given by SEM, where:
SEM=σn

In [ ]:
def calc_SEM(data):
    """
    Calculate Standard error of the mean. No nan's 
    should be in the input (numpy) array.
    """
    return np.std(data)/np.sqrt(len(data) - 1)


def gather_daily_stats(date, df):
    """
    For a specified day, given by date, create a short array of 
    observed values (obs) excluding the NANs. Return the mean, 
    and SEM value.
    Restrictions: more than one observation on a day, not a missing
    value, less than 300 mm per day (which is erroneous).
    """
    obs = np.array([df_prcp[key][day] for key in df_prcp.keys()])
    obs = obs[(obs > -1) & (obs < 500)]
    if len(obs) < 2:
        return np.NAN, np.NAN
    return np.mean(obs), calc_SEM(obs)


In [ ]:
#MAD based outlier calculation.
#def rej_Olier(data, thresh = 0.):
#    """
#    Calculate biweights of mean to reject outliers in df_prcp. No nan's 
#    should  also be in the input (numpy) array.
#    """
#    diff = np.abs(data - np.median(data))
#    mad = np.median(diff)   #median of the absolute deviation
#    mod_obs = diff/mad if mad else 0.
#    return data[mod_obs > thresh]

In [ ]:
# Create an accumulated time series (with SEM uncertainty values)
means = []
sems = []
for day in df_prcp.index:
    tmp_mean, tmp_sem = gather_daily_stats(date=day, df=df_prcp)
    means.append(tmp_mean)
    sems.append(tmp_sem)
means = np.array(means)
sems = np.array(sems)
df_prcp['Accumulated']=pd.Series(means,index=df_prcp.index)  #adding columns to the dataframe!
df_prcp['Acc_SEM']=pd.Series(sems,index=df_prcp.index)

In [ ]:
daily_ts = plt.figure(dpi=72)
daily_ts.set_size_inches(15,5)      # Specify the figure size
ax1 = daily_ts.add_subplot(111)     # Add an axis frame object to the plot (i.e. a pannel)

ax1.plot(df_prcp.index, df_prcp.Accumulated,'.g',ms=2.0)

ax1.errorbar(df_prcp.index, df_prcp.Accumulated,
             yerr=df_prcp.Acc_SEM, alpha=0.25, fmt=',')
ax1.set_ylim(0,80)
plt.xlim('1955-01-01','2015-08-31')
plt.title("Mean East African Precipitation")
plt.ylabel("Precipitation (mm day$^{-1}$)")
plt.xlabel("Year")
plt.grid(True)
plt.show(daily_ts)
#daily_ts.savefig('Daily_ts.pdf',dpi=300)

###3. Density plots###

In [ ]:
# N.b. the KDE (kernel density estimate) is Gaussian - which is not true
# for precip data (log or power law data)...
mask = df_prcp.Accumulated > 0.0
daily_dp = plt.figure()
daily_dp.set_size_inches(12,5)
ax = daily_dp.add_subplot(122)

sns.distplot(df_prcp.Accumulated[mask],bins=1000,norm_hist = True,kde=False,color = 'r')
sns.kdeplot(df_prcp.Accumulated[mask],shade=True,kernel='cos',cumulative=False,color='b')
leg1=ax.legend(['Cosine fit'],prop={'size':11},
                numpoints=1,markerscale=5.,frameon=True,fancybox=True)

ax.set_xlim(0,25)
ax.set_title("East Africa Mean Precipitation")
ax.set_xlabel(r'Precip. (mm day$^{-1}$)')
ax.set_ylabel('Density (0-1)')

plt.show(daily_dp)

In [ ]:
sns.kdeplot # Hot tip - look in SEABORN for statistical plots and help...

Tasks:
1. find out why the later part of the data has high variability
2. make sure you are happy/add any logical restrictions to improve the data quality in Accumulated dataset
3. Caclulate population statistics, histrogram, density plots (PDF, CDF), and fits to the population. Try several fit approaches, and show which is best.
4. Use the CDF (or a percentile function) to determine the key (IQR, median, tails etc) of the population
5. (hard) try to fit to the population. Reccomend trying a nth order polyfit using np.polyfit()
6. Use the statistical threshold values to define 'extreme' precipitation, and work out the:
  * frequency of extreme events,
  * duration (lenght) of extreme events,
  * magnitude (intensity) of extreme events
  
For task 6, you can plot these statistics as time dependent, or distributions, or something else...

In [ ]:
#plt.hist(df_prcp.Accumulated[mask], bins=60)
#plt.show()

In [ ]:
#Histogram
#source code from https://github.com/benlaken/Tanzania/blob/master/Precipitation_Tanzania.ipynb
hist_dp = plt.figure()
hist_dp.set_size_inches(5,5)          # Specify the output size
ax1 = hist_dp.add_subplot(211)        # Add an axis frame object to the plot (i.e. a pannel)
ax2 = hist_dp.add_subplot(212)        # Add an axis frame object to the plot (i.e. a pannel)

# the histogram of the data
ax1.set_title(r' Mean East African Precipitation')
n, bins, patches = ax1.hist(df_prcp.Accumulated[mask], 100, normed=True, facecolor='blue', alpha=0.75,
                            histtype='stepfilled')
ax1.grid(True)
ax1.set_ylabel('Density')
n, bins, patches = ax2.hist(df_prcp.Accumulated[mask], 100, normed=True, facecolor='blue', alpha=0.75,
                            histtype='stepfilled',cumulative=True)
plt.xlabel(r'mm day$^{-1}$')
plt.ylabel('Cumulative density')
plt.grid(True)

plt.show()
#hist_dp.savefig('Density_plots.pdf',dpi=300)

In [122]:
   # define extreme quantiles
percentileZero    = min(df_prcp.Accumulated[mask])
percentileHundred = max(df_prcp.Accumulated[mask])

print('Min. precip', percentileZero)
print('Max. precip', percentileHundred)
print("Median", np.percentile(df_prcp.Accumulated[mask],50))

Min. precip 0.025
Max. precip 52.55
Median 2.75


In [ ]:
srtd = sorted(df_prcp.Accumulated[mask])
percent = [val/len(srtd) * 100. for val in range(len(srtd))]
plt.plot(percent,srtd)
plt.grid(True)

In [ ]:
print(np.percentile(df_prcp.Accumulated[mask],90))
print(np.percentile(srtd,10))

###4. Seasonality###

Calculate the DOY mean over the data-period (climatology).

In [ ]:
doy_mean=[]
doy_sem =[]

for doy in range(366):
    index = df_prcp.index.dayofyear == doy+1 
    doy_mean.append(np.nanmean(df_prcp['Accumulated'][index]))
    doy_sem.append(calc_SEM(df_prcp['Accumulated'][index]))

doy_mean = np.array(doy_mean)
doy_sem = np.array(doy_sem)

In [ ]:
#Plot the seasonal climatology East Africa precip data
plt.errorbar(range(366),doy_mean,xerr=None, yerr=doy_sem, alpha=0.6)
plt.xlim(0,max(range(366)))
plt.title("East Africa DOY Mean ($\mu$) Rainfall")
plt.ylabel("Precipitation (mm)")
plt.xlabel("Day of Year (DOY)")
plt.grid(True)  

Anomaly
  * Use the seasonal DOY mean to calculate deviations (anomaly) from the daily mean

In [123]:
# wordy example of how to access/calculate anomaly
for daily_rain in zip(df_prcp.index[5000:5003],df_prcp.Accumulated[5000:5003]):
    print('Day {0}, rainfall {1:3.2f}mm'.format(daily_rain[0].date(),daily_rain[1]))
    print('DOY is',daily_rain[0].dayofyear)
    print("DOY climo value is {0:3.2f}".format(doy_mean[daily_rain[0].dayofyear -1]))
    print("Daily anomaly is {0:3.2f}".format(daily_rain[1] - doy_mean[daily_rain[0].dayofyear -1]))
    print(np.isnan(daily_rain[1]))
    print("")

Day 1954-12-16, rainfall 0.27mm
DOY is 350
DOY climo value is 2.16
Daily anomaly is -1.89
False

Day 1954-12-17, rainfall 7.63mm
DOY is 351
DOY climo value is 3.63
Daily anomaly is 4.01
False

Day 1954-12-18, rainfall 6.27mm
DOY is 352
DOY climo value is 2.16
Daily anomaly is 4.10
False



In [ ]:
#---Create a seasonal deviation from climatology--
#Anomalies = Observation - Climatology
prcp_anom = []
for daily_rain in zip(df_prcp.index,df_prcp.Accumulated):
    if np.isnan(daily_rain[1]):
        prcp_anom.append(np.NAN)
    else:
        prcp_anom.append(daily_rain[1] - doy_mean[daily_rain[0].dayofyear -1])
prcp_anom = np.array(prcp_anom)

In [ ]:
df_prcp['Acc_anomaly'] = prcp_anom  #adding columns to the dataframe!

In [ ]:
#plt.plot(df_prcp.index[prcp_anom > -999.],prcp_anom[prcp_anom > -999.],alpha=0.5)
#df_prcp

In [ ]:
# ---plot the anomalized rainfall data with errors---
my_anom = plt.figure(dpi=72)
my_anom.set_size_inches(15,6)        # Specify the output size
ax1 = my_anom.add_subplot(111)        # Add an axis frame object to the plot (i.e. a pannel)

ax1.errorbar(df_prcp['Acc_anomaly'].index,df_prcp['Acc_anomaly'],yerr=df_prcp['Acc_SEM'],xerr=None,alpha=0.5)
ax1.set_ylim(-10,50)
plt.xlim('1955-01-01','1990-08-31')
ax1.set_title('Seasonal Anomalized Precipitation Climatology')
ax1.set_ylabel(r'Anomalized Precip')
ax1.set_xlabel('Years')
ax1.grid(True)

#plt.legend(framealpha=0.9)
plt.show(my_anom)
#my_anom.savefig('EA anomalized.pdf',dpi=300)

In [ ]:
#doy_values = [doy.dayofyear - 1 for doy in df_prcp.index]
figx = plt.figure(dpi=72)
figx.set_size_inches(10,10)      # Specify the figure size
ax1 = figx.add_subplot(111)   

#---Plot the seasonal climatology East Africa precip data---
ax1.errorbar(range(366),doy_mean,xerr=None, yerr=doy_sem, alpha=0.6, )
ax1.plot(df_prcp.index.dayofyear -1 ,df_prcp['Accumulated'],'.',ms=2.5,alpha=1.0,color='r')
plt.xlim(0,max(range(366)))
plt.title("East Africa DOY Mean ($\mu$) Rainfall")
plt.ylabel("Precipitation (mm)")
plt.xlabel("Day of Year (DOY)")
plt.grid(True)

In [121]:
print('90th percentile = ',np.percentile(df_prcp['Acc_anomaly'][mask],90))
print('10th percentile = ',np.percentile(df_prcp['Acc_anomaly'][mask],10))
print('50th percentile = ',np.percentile(df_prcp['Acc_anomaly'][mask],50))
#sns.distplot(df_prcp['Acc_anomaly'][mask])

90th percentile =  6.41169871795
10th percentile =  -3.38829959514
50th percentile =  -0.608114035088


In [ ]:
my_dist = plt.figure()
my_dist.set_size_inches(15,5)               # Specify the output size
ax1 = my_dist.add_subplot(121)              # Add an axis frame object to the plot (i.e. a pannel)


sns.distplot(df_prcp['Acc_anomaly'][mask],bins=100, norm_hist=True, kde=False) # Filled bars  
sns.kdeplot(df_prcp['Acc_anomaly'][mask],shade=True,kernel='gau',cumulative=False,color='r')
ax1.vlines(np.percentile(df_prcp['Acc_anomaly'][mask],90), 0.00, 0.2, colors='b',linestyle='dashed')
ax1.vlines(np.percentile(df_prcp['Acc_anomaly'][mask],50), 0.00, 0.2, colors='b') # Marker line of Median
ax1.vlines(np.percentile(df_prcp['Acc_anomaly'][mask],10), 0.00, 0.2, colors='b',linestyle='dashed')
leg1=ax1.legend(['Gaussian fit'],prop={'size':11},numpoints=1,markerscale=5.,frameon=True,fancybox=True)

ax1.grid(True)
ax1.set_ylabel(r'Density',fontsize=11)
ax1.set_xlabel('Accumulated anomaly',fontsize=11)
ax1.set_xlim(-15, 40)
ax1.set_ylim(0.00, 0.2)


plt.show(my_dist)
#plt.savefig("EA Normalized precip.png",dpi=100,transparent=True)

Correlation Matrix

In [ ]:
corr_matrix = df_prcp
corr_matrix.corr(method='pearson', min_periods=1)

calculate the percentiles

In [ ]:
def percentile(df_prcp, percentile):
 """
Find the correspoding percentile of each value relative to a list of values.
where df is the list of values
 """
    p90 = []
    p95 = []
    p99 = [] 
    for day in df_prcp.index:
        tmp = np.array([df_prcp[key][day] for key in df_prcp.keys()])
        p90.append(np.percentile(tmp,90))    # ...find the nth percentile uncertainty values 
        p95.append(np.percentile(tmp,95)) 
        p99.append(np.percentile(tmp,99)) 
        
        
        return p90, p95, p99